In [ ]:
import sys
import os
import tempfile

from openghg.modules import ObsSurface
from openghg.processing import search
from pathlib import Path
from glob import glob

In [ ]:
tmp_dir = tempfile.TemporaryDirectory()
os.environ["OPENGHG_PATH"] = str(tmp_dir.name)

Here we'll use the test data to show how the ranking system works

In [ ]:
bsd_files = glob("../../../tests/data/proc_test_data/CRDS/bsd*")
bsd_files

Process the files

In [ ]:
bsd_results = ObsSurface.read_file(filepath=bsd_files, data_type="CRDS", site="bsd", network="DECC")

In [ ]:
bsd_results

Using the `Datasource` UUIDs we set the rank for each over a specific daterange, not that these cannot overlap

In [ ]:
uid_42 = bsd_results["processed"]["bsd.picarro.1minute.42m.min.dat"]["ch4"] 
uid_108 =  bsd_results["processed"]["bsd.picarro.1minute.108m.min.dat"]["ch4"]
uid_248 = bsd_results["processed"]["bsd.picarro.1minute.248m.min.dat"]["ch4"]

obs = ObsSurface.load()
obs.set_rank(uuid=uid_248, rank=1, date_range="2012-01-01_2013-01-01")
obs.set_rank(uuid=uid_108, rank=1, date_range="2014-09-02_2014-11-01")
obs.set_rank(uuid=uid_248, rank=1, date_range="2015-01-01_2015-11-01")
obs.set_rank(uuid=uid_108, rank=1, date_range="2016-09-02_2018-11-01")
obs.set_rank(uuid=uid_42, rank=1, date_range="2019-01-02_2021-01-01")

We can have a look at the ranking data stored in `ObsSurface`

In [ ]:
obs._rank_data

In [ ]:
search_res = search(site="bsd", species="ch4")


If we print the object we get the results in an easily readable format

In [ ]:
print(search_res)

The `__repr__` method that the notebook uses prints the whole raw dictionary

In [ ]:
search_res

This can also be accessed using

In [ ]:
search_res.raw()

We can retrieve the metadata associated with a species at a site

In [ ]:
metadata = search_res.metadata(site="bsd", species="ch4")
metadata

This tells us the inlet height we've retrieved data for over specific dateranges (this interface will be improved)

In [ ]:
rank_metadata = search_res.raw()["bsd"]["ch4"]["rank_metadata"]
rank_metadata

We can retrieve the `ObsData` object using the `retrieve` method on the `SearchResults` object

In [ ]:
data = search_res.retrieve(site="bsd", species="ch4")
data

And then get the xarray Dataset using

In [ ]:
data.data

We can also get the rank metadata using

In [ ]:
data.metadata["rank_metadata"]